In [2]:
#Install packages
!pip install transformers
#!pip install beautifulsoup4 pandas selenium
#!pip install lxml
#!pip install html5lib
#!pip install requests

     |████████████████████████████████| 3.8 MB 4.5 MB/s 
     |████████████████████████████████| 596 kB 38.7 MB/s 
     |████████████████████████████████| 6.5 MB 51.2 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
     |████████████████████████████████| 895 kB 46.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
## Import Libraries
from transformers import pipeline
from bs4 import BeautifulSoup
import requests


In [4]:
#Source URL
URL="https://ai.googleblog.com/2022/03/auto-generated-summaries-in-google-docs.html"

In [5]:
## Request for the website
source=requests.get(URL).text


In [6]:
##BeautifulSoup -For parsing webpage
soup=BeautifulSoup(source,'html')
#print(soup.prettify())

In [7]:
#Extract titles and text 
results=soup.find_all(['h2','b','p'])
#results

In [9]:
# Remove label
text=[result.text for result in results]
Article=' '.join(text)
Article

'\n\nAuto-generated Summaries in Google Docs\n\n \nFor many of us, it can be challenging to keep up with the volume of documents that arrive in our inboxes every day: reports, reviews, briefs, policies and the list goes on. When a new document is received, readers often wish it included a brief summary of the main points in order to effectively prioritize it. However, composing a document summary can be cognitively challenging and time-consuming, especially when a document writer is starting from scratch.\n \nTo help with this, we recently announced that Google Docs now automatically generates suggestions to aid document writers in creating content summaries, when they are available. Today we describe how this was enabled using a machine learning (ML) model that comprehends document text and, when confident, generates a 1-2 sentence natural language description of the document content. However, the document writer maintains full control — accepting the suggestion as-is, making necessar

**Chunk Article**

In [11]:
## Modify punctuations
Article =Article .replace('.','.<eos>')
Article =Article .replace('!','!<eos>')
Article =Article .replace('?','?<eos>')
sentences=Article.split('<eos>')


In [9]:
sentences

['\n            Blog\n           \n\nAuto-generated Summaries in Google Docs\n\n \nFor many of us, it can be challenging to keep up with the volume of documents that arrive in our inboxes every day: reports, reviews, briefs, policies and the list goes on.',
 ' When a new document is received, readers often wish it included a brief summary of the main points in order to effectively prioritize it.',
 ' However, composing a document summary can be cognitively challenging and time-consuming, especially when a document writer is starting from scratch.',
 '\n \nTo help with this, we recently announced that Google Docs now automatically generates suggestions to aid document writers in creating content summaries, when they are available.',
 ' Today we describe how this was enabled using a machine learning (ML) model that comprehends document text and, when confident, generates a 1-2 sentence natural language description of the document content.',
 ' However, the document writer maintains full 

In [12]:
#Define Chunk size
max_chunk=500
current_chunk=0
chunks=[]

for sentence in sentences:
  if len(chunks)==current_chunk +1:
    if len(chunks[current_chunk])+len(sentence.split(' '))<=max_chunk:
      chunks[current_chunk].extend(sentence.split(' '))
    else:
        current_chunk += 1
        chunks.append(sentence.split(' '))
  else:
          print(current_chunk)
          chunks.append(sentence.split(' '))

0


In [13]:
#Join the chunked word 
for chunk_id in range(len(chunks)):
  chunks[chunk_id]=' '.join(chunks[chunk_id])

In [14]:
chunks[1]

'\n \nThe Pegasus work took this idea one step further, by introducing a pre-training objective customized to abstractive summarization.  In Pegasus pre-training, also called Gap Sentence Prediction (GSP), full sentences from unlabeled news articles and web documents are masked from the input and the model is required to reconstruct them, conditioned on the remaining unmasked sentences.  In particular, GSP attempts to mask sentences that are considered essential to the document through different heuristics.  The intuition is to make the pre-training as close as possible to the summarization task.  Pegasus achieved state-of-the-art results on a varied set of summarization datasets.  However, a number of challenges remained to apply this research advancement into a product. \n \nApplying Recent Research Advances to Google Docs\n\n Applying Recent Research Advances to Google Docs \n\n    Self-supervised pre-training results in an ML model that has general language understanding and genera

In [15]:
len(chunks[1].split(' '))

497

**Summarization Task**

In [16]:
#Load summarizationn pipeline
summarizer=pipeline('summarization')


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [17]:
##Summmarize chunks
res=summarizer(chunks,max_length=80,min_length=30,do_sample=False)
res

[{'summary_text': ' Google Docs now automatically generates suggestions to aid document writers in creating content summaries, when they are available . Auto-generated suggestions are currently only available to Google Workspace business customers . Building on grammar suggestions, Smart Compose, and autocorrect, we see this as another valuable step toward improving written communication in the workplace .'},
 {'summary_text': ' The Pegasus work took this idea one step further, by introducing a pre-training objective customized to abstractive summarization . Pegasus achieved state-of-the-art results on a varied set of summarization datasets . However, a number of challenges remained to apply this research advancement into a product .'},
 {'summary_text': ' We used knowledge distillation to distill Pegasus model into a hybrid architecture of a Transformer encoder and an RNN decoder . The resulting model had significant improvements in latency and memory footprint while the quality was s

In [18]:
# Output to Text file
Text = ' '.join([summ['summary_text'] for summ in res])
Text

' Google Docs now automatically generates suggestions to aid document writers in creating content summaries, when they are available . Auto-generated suggestions are currently only available to Google Workspace business customers . Building on grammar suggestions, Smart Compose, and autocorrect, we see this as another valuable step toward improving written communication in the workplace .  The Pegasus work took this idea one step further, by introducing a pre-training objective customized to abstractive summarization . Pegasus achieved state-of-the-art results on a varied set of summarization datasets . However, a number of challenges remained to apply this research advancement into a product .  We used knowledge distillation to distill Pegasus model into a hybrid architecture of a Transformer encoder and an RNN decoder . The resulting model had significant improvements in latency and memory footprint while the quality was still on par with the original model . The authors would like t

In [17]:
#Output File save
#with open('blogsummary.txt','w') as f:
       #f.write(Text)